In [ ]:
%load_ext autoreload
%autoreload 2
%aimport utils_1_1

import pandas as pd
import numpy as np
import altair as alt
from altair_saver import save
import datetime
import dateutil.parser
from os.path import join

from constants_1_1 import SITE_FILE_TYPES
from utils_1_1 import (
    get_site_file_paths,
    get_site_file_info,
    get_site_ids,
    get_visualization_subtitle,
    get_country_color_map,
)
from theme import apply_theme
from web import for_website

alt.data_transformers.disable_max_rows(); # Allow using rows more than 5000

consistent_loinc = {
    "C-reactive protein (CRP) (Normal Sensitivity)": "C-reactive protein (Normal Sensitivity) (mg/dL)",
    "creatinine": "Creatinine (mg/dL)",
    "Ferritin": "Ferritin (ng/mL)",
    "D-dimer": "D-dimer (ng/mL)",
    "albumin": "Albumin (g/dL)",        

    "Fibrinogen": "Fibrinogen (mg/dL)",
    "alanine aminotransferase (ALT)": "Alanine aminotransferase (U/L)",
    "aspartate aminotransferase (AST)": "Aspartate aminotransferase (U/L)",
    "total bilirubin": "Total bilirubin (mg/dL)",
    "lactate dehydrogenase (LDH)": "Lactate dehydrogenase (U/L)",
    "cardiac troponin": "Cardiac troponin (ng/mL)",
    "prothrombin time (PT)": "Prothrombin time (s)",
    "white blood cell count (Leukocytes)": "White blood cell count (10*3/uL)",
    "lymphocyte count": "Lymphocyte count (10*3/uL)",
    "neutrophil count": "Neutrophil count (10*3/uL)",
    "procalcitonin": "Procalcitonin (ng/mL)",
}

In [ ]:
df = pd.read_csv(join("..", "data", "1.1.resurgence", "labs", "lab_values_standardized.csv"))

df

In [ ]:
obs = pd.read_csv(join("..", "data", "1.1.resurgence", "labs", "lab_obs.csv"))

obs

In [ ]:
pdf = pd.read_csv(join("..", "data", "1.1.resurgence", "labs", "lab_pvals.csv"))

pdf

In [ ]:
def plot(_d, patient_group='all', lab=None, obs=None, pdf=None):
    d = _d.copy()
    o = obs.copy()
    p = pdf.copy()
    
    """
    DATA PREPROCESSING...
    """
    if lab in ['C-reactive protein (CRP) (Normal Sensitivity)', 'D-dimer']:
        d = d[d.scale == 'log']
        p = p[p.scale == 'log']
    else:
        d = d[d.scale == 'original']
        p = p[p.scale == 'original']
        
    d = d.drop(columns=['Unnamed: 0'])
    d.wave = d.wave.apply(lambda x: { 'early': 'Early', 'late': 'Late' }[x])
    d = d[d.setting != 'never']
    d = d[d.setting == patient_group]
    d = d.rename(columns={
        'wave': 'Wave'
    })
    
    d = d[d.Lab == lab]
    
    d17 = d[(1 <= d.days_since_positive) & (d.days_since_positive <= 7)]
    d = d[(d.days_since_positive == 0) | (d.days_since_positive == 1) | (d.days_since_positive == 7)]

    d.days_since_positive = d.days_since_positive.apply(lambda x: f"Day {x}")
    d17.days_since_positive = d17.days_since_positive.apply(lambda x: f"Day {x}")
    
    #### PVAL ###############################
    p = p[p.setting == patient_group]
    p['is_sig'] = False
    p.is_sig = p[lab] <= 0.05
    p.is_sig = p.is_sig.apply(lambda x: 'p<0.05' if x else 'p>0.05')
    p = p[['setting','day', 'is_sig']]
    p.day = p.day.apply(lambda x: f"Day {x}")
    
    """
    MERGE
    """
    d = pd.merge(d, p, how='left', left_on=['days_since_positive','setting'], right_on = ['day','setting'])
    
    #### OBS ################################
    o = o[o.lab == lab]
    o = o.drop(columns=['Unnamed: 0'])
    o.wave = o.wave.apply(lambda x: { 'early': 'Early', 'late': 'Late' }[x])
    o = o[o.cohort == 'dayX']
    o = o[o.setting == patient_group]
    o = o.rename(columns={
        'wave': 'Wave'
    })    
    
    """
    DAY 1-7 AVERAGE
    """
    d17['mean'] = d17['mean'] * d17['total_n']
    d17 = d17.groupby(['Lab', 'setting', 'Wave']).sum().reset_index()
    d17['mean'] = d17['mean'] / d17['total_n']

    """
    CONSTANTS
    """
    LABS = d.Lab.unique().tolist()
    WAVE_COLOR = [
        '#D45E00', # '#BA4338', # early
        '#0072B2', # late
        'black'
    ]
    
    """
    PLOT
    """
    titleX=-60    
    opacity=0.7
    
    """
    LABS
    """
    LAB_DROPDOWN = alt.binding_select(options=LABS)
    LAB_SELECTION = alt.selection_single(fields=["Lab"], bind=LAB_DROPDOWN, init={"Lab": lab if lab != None else LABS[0]}, name="Select")
    
    line_m = alt.Chart(
        d
    ).mark_line(
        size=4, opacity=opacity, point=False
    ).encode(
        x=alt.X('days_since_positive:N', title=None), # ''Days Since Positive'),
        y=alt.Y('mean:Q', scale=alt.Scale(zero=False), title='Mean Lab Value', axis=alt.Axis(titleX=titleX)),
        color=alt.Color('Wave:N', scale=alt.Scale(domain=['Early', 'Late'], range=WAVE_COLOR))
    ).properties(
        width=200,
        height=200
    )
    
    point_m = alt.Chart(
        d
    ).mark_point(
        opacity=opacity, filled=True, strokeWidth=3
    ).encode(
        x=alt.X('days_since_positive:N', title=None), # ''Days Since Positive'),
        y=alt.Y('mean:Q', scale=alt.Scale(zero=False), title='Mean Lab Value', axis=alt.Axis(titleX=titleX)),
        color=alt.Color('Wave:N', scale=alt.Scale(range=WAVE_COLOR)),
        stroke=alt.Stroke('is_sig:N', scale=alt.Scale(domain=['p<0.05'], range=['black']), title='Significance'),
        size=alt.Size('total_n:Q', title="# of Patients"),
        strokeWidth=alt.value(3)
    )
    
    line_m = (line_m + point_m)
#     .add_selection(
#         LAB_SELECTION
#     ).transform_filter(
#         LAB_SELECTION
#     )
    
    """
    Day 1-7 Average
    """
    bar_m = alt.Chart(
        d17
    ).mark_bar(
        size=32,
        stroke='black'
    ).encode(
        x=alt.X('Wave:N', title=None), # 'Wave'),
        y=alt.Y('mean:Q', title='Day1-7 Mean Lab Value', axis=alt.Axis(ticks=False, labels=False, domain=False, orient='left')),
        color=alt.Color('Wave:N', scale=alt.Scale(range=WAVE_COLOR))
    ).properties(
        width=100
    )
#     .add_selection(
#         LAB_SELECTION
#     ).transform_filter(
#         LAB_SELECTION
#     )
    
    text = alt.Chart(
        d17
    ).mark_text(size=16, dx=0, dy=-4, color='black', baseline='bottom', align='center', angle=0, fontWeight=500).encode(
        x=alt.X('Wave:N', title=None), # 'Wave'),
        y=alt.Y('mean:Q', title='Day1-7 Mean Lab Value', axis=alt.Axis(ticks=False, labels=False, domain=False, orient='left')),
        text=alt.Text('mean:Q', format=".2f")
    )
#     .transform_filter(
#         LAB_SELECTION
#     )
    bar_m = (bar_m)# + text)
       
        
    """
    OBSERVATION
    """
    LAB_FIELD_NAME = lab.replace(' ', '.').replace('(', '.').replace(')', '.').replace('-', '.') # Used in the original files
    o = o.rename(columns={
        f"{LAB_FIELD_NAME}": LAB_FIELD_NAME.replace('.', '_')
    })
    LAB_FIELD_NAME = LAB_FIELD_NAME.replace('.', '_')
    
    obs_line = alt.Chart(
        o
    ).mark_line(
        point=True, opacity=0.7, size=4
    ).encode(
        x=alt.X('day:Q', title='Days Since Positive'),
        y=alt.Y(f"p:Q", axis=alt.Axis(format='0.0%', titleX=titleX), title="% Patients"),
        color=alt.Color("Wave:N")
    ).properties(
        title={
            "text": "Percentage of Patients Tested",
            "dx": 80,
            # "fontSize": 16,
            # "color": "gray"
        },
        width=350,
        height=150
    )
    
    plot = (
        alt.vconcat(
            alt.hconcat(line_m, bar_m, spacing=20).resolve_scale(y='shared'),
            obs_line,
            spacing=20
        ).resolve_scale(x='independent')
    ).properties(
            title={
                "text": [
                    f"Mean Lab Values of {'All' if patient_group == 'all' else 'Ever Severe'} Patients by Wave"
                ],
                "dx": 80,
                "subtitle": [
                    consistent_loinc[lab], #.title(),
                    get_visualization_subtitle(data_release='2021-02-15', with_num_sites=False)], 
                "subtitleFontSize": 16,
                "subtitleColor": "gray",
            }
        )
    
    return plot

In [ ]:
LABS = df.Lab.unique().tolist()

for i, lab in enumerate(LABS):
    new = alt.hconcat(
        plot(df, patient_group='all', lab=lab, obs=obs, pdf=pdf), 
        plot(df, patient_group='ever', lab=lab, obs=obs, pdf=pdf), 
        spacing=50
    ).resolve_scale(y='shared', color='independent', size='independent', stroke='independent')
    if i != 0:
        res = alt.vconcat(res, new, spacing=30)
    else:
        res = new
    # DEBUG
    #if i == 0:
        #break

res = apply_theme(
    res,
    axis_y_title_font_size=16,
    title_anchor='start',
    legend_orient='right'
)
res

In [ ]:
cdf = pd.read_csv(join("..", "data", "1.1.resurgence", "labs", "lab_values_standardized_bycountry.csv"))

cdf

In [ ]:
cobs = pd.read_csv(join("..", "data", "1.1.resurgence", "labs", "lab_obs_bycountry.csv"))

cobs

In [ ]:
fr = pd.read_csv(join("..", "data", "1.1.resurgence", "labs", "lab_pvals_FRANCE.csv"))
ge = pd.read_csv(join("..", "data", "1.1.resurgence", "labs", "lab_pvals_GERMANY.csv"))
it = pd.read_csv(join("..", "data", "1.1.resurgence", "labs", "lab_pvals_ITALY.csv"))
sp = pd.read_csv(join("..", "data", "1.1.resurgence", "labs", "lab_pvals_SPAIN.csv"))
us = pd.read_csv(join("..", "data", "1.1.resurgence", "labs", "lab_pvals_USA.csv"))

cpdf = fr.append([ge, it, sp, us])
cpdf = cpdf.drop(columns=['Unnamed: 0'])
cpdf

In [ ]:
def plot_country(_d, patient_group='all', lab=None, obs=None, pdf=None):
    d = _d.copy()
    o = obs.copy()
    p = pdf.copy()
    
    """
    DATA PREPROCESSING...
    """
    if lab in ['C-reactive protein (CRP) (Normal Sensitivity)', 'D-dimer']:
        d = d[d.scale == 'log']
        p = p[p.scale == 'log']
    else:
        d = d[d.scale == 'original']
        p = p[p.scale == 'original']
        
    d.wave = d.wave.apply(lambda x: { 'early': 'Early', 'late': 'Late' }[x])
    d = d[d.setting != 'never']
    d = d[d.setting == patient_group]
    d = d.rename(columns={
        'wave': 'Wave'
    })
    d = d[d.country != 'SINGAPORE']
    d = d[d.country != 'GERMANY']
    d.country = d.country.apply(lambda x: x.capitalize() if x != 'USA' else x)
    d = d[(d.days_since_positive == 0) | (d.days_since_positive == 1) | (d.days_since_positive == 7)]
    d.days_since_positive = d.days_since_positive.apply(lambda x: f"Day {x}")
    d = d[d.Lab == lab]

    """
    OBS DATA
    """
    # o = o.drop(columns=['Unnamed: 0'])
    o = o[o.lab == lab]
    o.wave = o.wave.apply(lambda x: { 'early': 'Early', 'late': 'Late' }[x])
    o = o[o.cohort == 'dayX']
    o = o[o.country != 'GERMANY']
    o.country = o.country.apply(lambda x: x.capitalize() if x != 'USA' else x)
    o = o[o.setting == patient_group]
    o = o.rename(columns={
        'wave': 'Wave'
    })    
    
    """
    PVAL DATA
    """
    p = p[p.setting == patient_group]
    p.country = p.country.apply(lambda x: x.capitalize() if x != 'USA' else x)
    p['is_sig'] = False
    p.is_sig = p[lab] <= 0.05
    p.is_sig = p.is_sig.apply(lambda x: 'p<0.05' if x else 'p>0.05')
    p = p[['setting','day', 'country', 'is_sig']]
    p.day = p.day.apply(lambda x: f"Day {x}")
    
    """
    MERGE
    """
    d = pd.merge(d, p, how='left', left_on=['days_since_positive','setting', 'country'], right_on = ['day','setting', 'country'])
    
    """
    CONSTANTS
    """
    LABS = d.Lab.unique().tolist()
    WAVE_COLOR = [
        '#D45E00', # '#BA4338', # early
        '#0072B2', # late
        'black'
    ]
    COUNTRY_COLORS = ['#0072B2', '#029F73', '#D45E00', '#CB7AA7'] # '#E79F00', 
    
    """
    PLOT
    """
    titleX=-60    
    opacity=0.7
    
    """
    LABS
    """
    LAB_DROPDOWN = alt.binding_select(options=LABS)
    LAB_SELECTION = alt.selection_single(fields=["Lab"], bind=LAB_DROPDOWN, init={"Lab": LABS[0]}, name="Select")
    
    line_m = alt.Chart(
        d
    ).mark_line(
        size=4, opacity=opacity
    ).encode(
        x=alt.X('days_since_positive:N', title=None),
        y=alt.Y('mean:Q', scale=alt.Scale(zero=False), title='Standardized Mean Lab', axis=alt.Axis(titleX=titleX)),
        color=alt.Color('country:N', scale=alt.Scale(range=COUNTRY_COLORS))
    ).properties(
        width=200,
        height=200
    )
    
    point_m = line_m.mark_point(
        size=80, opacity=opacity, filled=True
    ).encode(
        x=alt.X('days_since_positive:N', title=None),
        y=alt.Y('mean:Q', scale=alt.Scale(zero=False), title='Standardized Mean Lab', axis=alt.Axis(titleX=titleX)),
        color=alt.Color('country:N', scale=alt.Scale(range=COUNTRY_COLORS)),
#         stroke=alt.Stroke('is_sig:N', scale=alt.Scale(domain=['p<0.05'], range=['black']), title='Significance'),
#         strokeWidth=alt.value(3)
    )
    
    line_m = (line_m + point_m).facet(
        column=alt.Column('Wave:N', header=alt.Header(labelOrient="top", title=None, titleOrient="bottom", labels=True))
    )
#     .add_selection(
#         LAB_SELECTION
#     ).transform_filter(
#         LAB_SELECTION
#     )
    
    """
    OBSERVATION
    """
    LAB_FIELD_NAME = lab.replace(' ', '.').replace('(', '.').replace(')', '.').replace('-', '.') # Used in the original files
    o = o.rename(columns={
        f"{LAB_FIELD_NAME}": LAB_FIELD_NAME.replace('.', '_')
    })
    LAB_FIELD_NAME = LAB_FIELD_NAME.replace('.', '_')

    obs_line = alt.Chart(
        o
    ).mark_line(
        point=True, opacity=0.7, size=4
    ).encode(
        x=alt.X('day:Q', title='Days Since Positive'),
        y=alt.Y(f"p:Q", axis=alt.Axis(format='0.0%', titleX=titleX), title="% Patients"),
        color=alt.Color('country:N', scale=alt.Scale(range=COUNTRY_COLORS)),
        column=alt.Column('Wave:N', header=alt.Header(labelOrient="top", title=None, titleOrient="top", labels=False))
    ).properties(
        width=200,
        height=100
    )
#     .properties(
#         title={
#             "text": "Percentage of Patients Tested",
#             "dx": 80,
#             # "fontSize": 16,
#             # "color": "gray"
#         },
#         width=350,
#         height=100
#     )
    
    line_m = alt.vconcat(line_m, obs_line, spacing=20)
    
    plot = (
        alt.hconcat(line_m, spacing=20).resolve_scale(y='shared')
    ).properties(
            title={
                "text": f"Country-Level Lab Values of {'All' if patient_group == 'all' else 'Ever Severe'} Patients by Wave",
                "dx": 80,
                "subtitle": [
                    consistent_loinc[lab], #.title(),
                    get_visualization_subtitle(data_release='2021-02-15', with_num_sites=False)
                ], 
                "subtitleFontSize": 16,
                "subtitleColor": "gray",
            }
        )
    
    return plot

In [ ]:
LABS = df.Lab.unique().tolist()
# print(len(LABS))

for i, lab in enumerate(LABS):
    new = alt.hconcat(
        plot_country(cdf, patient_group='all', lab=lab, obs=cobs, pdf=cpdf), 
        plot_country(cdf, patient_group='ever', lab=lab, obs=cobs, pdf=cpdf), 
        spacing=50
    ).resolve_scale(y='shared', color='independent', size='independent', stroke='independent')
    if i != 0:
        res = alt.vconcat(res, new, spacing=10)
    else:
        res = new
    # DEBUG
#    if i == 0:
#         break
res = apply_theme(
    res,
    axis_y_title_font_size=16,
    title_anchor='start',
    legend_orient='right',
    header_label_font_size=16
)
res

In [ ]:
def day0_overview(_d, lab, _p):
    d = _d.copy()
    p = _p.copy()
    
    """
    DATA PREPROCESSING...
    """
    d.wave = d.wave.apply(lambda x: { 'early': 'Early', 'late': 'Late' }[x])
    d = d.rename(columns={
        'wave': 'Wave'
    })
    d = d[d.country != 'SINGAPORE']
    d = d[d.country != 'GERMANY']
    d = d[d.days_since_positive == 0]
    d = d[d.Lab == lab]
    d.country = d.country.apply(lambda x: x.capitalize() if x != 'USA' else x)
    d.days_since_positive = d.days_since_positive.apply(lambda x: f"Day {x}")
    d.setting = d.setting.apply(lambda x: {'all': 'All Patients', 'ever': 'Ever Severe Patients', 'never': 'Never Severe Patients'}[x])
    
    if lab in ['C-reactive protein (CRP) (Normal Sensitivity)', 'D-dimer']:
        d = d[d.scale == 'log']
        p = p[p.scale == 'log']
    else:
        d = d[d.scale == 'original']
        p = p[p.scale == 'original']
    
    d = d.drop(columns=['scale', 'Lab', 'days_since_positive'])
    d = d.pivot_table(values='mean', index=['country', 'setting'], columns='Wave').reset_index()
    d = d[(d.Early.notna() & d.Late.notna())]
    d['increase'] = d.Early < d.Late
    
    p.country = p.country.apply(lambda x: x.capitalize() if x != 'USA' else x)
    p['sig'] = p[lab].apply(lambda x: 'yes' if x < 0.05 else 'no')
    p[lab] = -np.log10(p[lab])
    
    d = pd.merge(d, p,  how='left', left_on=['country','setting'], right_on = ['country','setting'])
    
    # print(d)
    # print(p[lab])
    # print(p)
    """
    CONSTANTS
    """
    COUNTRIES = ['France', 'Italy', 'Spain', 'USA'] # 'Germany', 
    COUNTRY_COLORS = ['#0072B2', '#029F73', '#D45E00', '#CB7AA7'] # '#E79F00', 
    
    """
    PLOT
    """
    titleX=-80
    opacity=0.7
    
    """
    LABS
    """    
    bar = alt.Chart(
        d
    ).mark_bar(
        size=6
    ).encode(
        y=alt.Y('Early:Q', scale=alt.Scale(zero=False), title='Mean Lab Value'),
        y2=alt.Y2('Late:Q'),
        x=alt.X('country:N', title='Country'), # axis=alt.Axis(titleX=titleX)),
        color=alt.Color('country:N', title='Country', scale=alt.Scale(domain=COUNTRIES, range=COUNTRY_COLORS))
    ).properties(
        width=200,
        height=220
    )
    
    tr = alt.Chart(
        d
    ).transform_filter(
        alt.FieldOneOfPredicate(field='increase', oneOf=[True])
    ).mark_point(
        shape="triangle-up", filled=True, size=300, yOffset=3, opacity=1
    ).encode(
        y=alt.Y('Late:Q', scale=alt.Scale(zero=False), title='Mean Lab Value'),
        x=alt.X('country:N', title='Country'),# axis=alt.Axis(titleX=titleX)),
        color=alt.Color('country:N', title='Country', scale=alt.Scale(domain=COUNTRIES, range=COUNTRY_COLORS))
    )
    
    tl = alt.Chart(
        d
    ).transform_filter(
        alt.FieldOneOfPredicate(field='increase', oneOf=[False])
    ).mark_point(
        shape="triangle-down", filled=True, size=300, yOffset=-3, opacity=1
    ).encode(
        y=alt.Y('Late:Q', scale=alt.Scale(zero=False), title='Mean Lab Value'),
        x=alt.X('country:N', title='Country'), # axis=alt.Axis(titleX=titleX)),
        color=alt.Color('country:N', title='Country', scale=alt.Scale(domain=COUNTRIES, range=COUNTRY_COLORS))
    )
    
    baseline = alt.Chart(
        pd.DataFrame({'baseline': [1]})
    ).mark_rule(color='gray').encode(
        y=alt.Y('baseline:Q')
    )
        
    plot = (bar + tr + tl).facet(
        column=alt.Column('setting:N', header=alt.Header(labelOrient="top", title=None, titleOrient="top", labels=True)),
        spacing=30
    )
#     .add_selection(
#         LAB_SELECTION
#     ).transform_filter(
#         LAB_SELECTION
#     )
    
    plot = plot.properties(
        title={
            "text": [
                f"Standardized Mean Lab Values by Country from Early to Late Wave"
            ],
            "dx": 50,
            "subtitle": [
                # 'Relative to Day 0 Mean Value During Early Phase',
                consistent_loinc[lab],
                #get_visualization_subtitle(data_release='2021-01-25', with_num_sites=False)
            ], 
            "subtitleFontSize": 16,
            "subtitleColor": "gray",
        }
    )
    
    """
    p-value triangles
    """
    d_notna = d[d[lab].notna()].copy()

    p_base = alt.Chart(
        d_notna
    ).mark_point(
        size=200, filled=True, opacity=1, shape='triangle-up', strokeWidth=1, xOffset=0
    ).encode(
        x=alt.X('country:N', title='Country'),# axis=alt.Axis(titleX=titleX)),
        y=alt.Y(f'{lab}:Q', title="P Value (-log10)", scale=alt.Scale(zero=False)),
        color=alt.Color('country:N', scale=alt.Scale(domain=COUNTRIES, range=COUNTRY_COLORS), title='Country'),
        stroke=alt.Stroke('sig:N', scale=alt.Scale(domain=['no', 'yes'], range=['white', 'black']), title='p < 0.05?', legend=None)
    ).properties(
        width=200,
        height=200
    )
    
    p_base_no_y = p_base.encode(
        x=alt.X('country:N', title='Country'),
        y=alt.Y(f'{lab}:Q', title="P Value (-log10)", scale=alt.Scale(zero=False), axis=alt.Axis(ticks=False, domain=False, title=None, labels=False)),
        color=alt.Color('country:N', scale=alt.Scale(domain=COUNTRIES, range=COUNTRY_COLORS)),
        stroke=alt.Stroke('sig:N', scale=alt.Scale(domain=['no', 'yes'], range=['white', 'black']), title='p < 0.05?', legend=None)
    )

    p0_05_all = alt.Chart(
        pd.DataFrame({'baseline': [1.30102999566], 'zero': [0]})
    ).mark_rule(color='firebrick', strokeDash=[3,3]).encode(
        y=alt.Y('baseline:Q')
    )
    
    p0_05_all_rect = alt.Chart(
        pd.DataFrame({'baseline': [1.30102999566]})
    ).mark_rect(color='transparent', stroke='firebrick', strokeDash=[3,3], strokeWidth=2).encode(
        y=alt.Y('baseline:Q'),
        y2=alt.value(0)
    )
    
    p0_05_text = alt.Chart(
        pd.DataFrame({'baseline': [1.30102999566], 'text': 'p=0.05'})
    ).mark_text(color='firebrick', align='right', baseline='top', y=0, x=195).encode(
        # y=alt.Y('baseline:Q'),
        y=alt.value(2),
        text=alt.value('Statistical Significance: p<0.05')
    )
    
    """
    ALL PATIENTS
    """    
    p_t_all1 = (
        #p0_05_all
        #+ 
        p0_05_text
        + 
        p_base.transform_filter(
            alt.FieldOneOfPredicate(field='increase', oneOf=[True])
        ).transform_filter(
            alt.FieldOneOfPredicate(field='setting', oneOf=['All Patients'])
        )
        + 
        p_base.mark_point(
            size=200, filled=True, opacity=1, shape='triangle-down', strokeWidth=1, xOffset=0
        ).transform_filter(
            alt.FieldOneOfPredicate(field='increase', oneOf=[False])
        ).transform_filter(
            alt.FieldOneOfPredicate(field='setting', oneOf=['All Patients'])
        ) 
        +
        p0_05_all_rect
    ).properties(
        title={
            'text': 'All Patients',
            'dx': 105,
            'fontSize': 16
        }
    )
    
    """
    EVER SEVERE
    """    
    p_t_all2 = (
        #p0_05_all 
        #+ 
        p0_05_text
        + 
        p_base_no_y.transform_filter(
            alt.FieldOneOfPredicate(field='setting', oneOf=['Ever Severe Patients'])
        ).transform_filter(
            alt.FieldOneOfPredicate(field='increase', oneOf=[True])
        )
        + 
        p_base_no_y.mark_point(
            size=200, filled=True, opacity=1, shape='triangle-down', strokeWidth=1, xOffset=0
        ).transform_filter(
            alt.FieldOneOfPredicate(field='setting', oneOf=['Ever Severe Patients'])
        ).transform_filter(
            alt.FieldOneOfPredicate(field='increase', oneOf=[False])
        )
        + p0_05_all_rect
    ).properties(
        title={
            'text': 'Ever Severe Patients',
            'dx': 30,
            'fontSize': 16
        }
    )
    
    """
    NEVER SEVERE
    """    
    p_t_all3 = (
        #p0_05_all 
        #+ 
        p0_05_text
        + 
        p_base_no_y.transform_filter(
            alt.FieldOneOfPredicate(field='setting', oneOf=['Never Severe Patients'])
        ).transform_filter(
            alt.FieldOneOfPredicate(field='increase', oneOf=[True])
        )
        + 
        p_base_no_y.mark_point(
            size=200, filled=True, opacity=1, shape='triangle-down', strokeWidth=1, xOffset=0
        ).transform_filter(
            alt.FieldOneOfPredicate(field='setting', oneOf=['Never Severe Patients'])
        ).transform_filter(
            alt.FieldOneOfPredicate(field='increase', oneOf=[False])
        )
        + p0_05_all_rect
    ).properties(
        title={
            'text': 'Never Severe Patients',
            'dx': 30,
            'fontSize': 16
        }
    )
    
    """
    COMBINE
    """
    p_t = alt.hconcat(p_t_all1, p_t_all2, p_t_all3, spacing=20).resolve_scale(y='shared')
    
    p_t = p_t.properties(
        title={
            "text": [
                f"P-Values at 0 Days Since Positive Across Waves"
            ],
            "dx": 50,
            ''
            "subtitle": [
                # 'Relative to Day 0 Mean Value During Early Phase',
                consistent_loinc[lab],
                #get_visualization_subtitle(data_release='2021-01-25', with_num_sites=False)
            ], 
            "subtitleFontSize": 16,
            "subtitleColor": "gray",
        }
    )
    
    
    return alt.vconcat(plot, p_t, spacing=30).resolve_scale(color='independent', stroke='independent')

fr = pd.read_csv(join("..", "data", "1.1.resurgence", "labs", "lab_pvals_FRANCE.csv"))
ge = pd.read_csv(join("..", "data", "1.1.resurgence", "labs", "lab_pvals_GERMANY.csv"))
it = pd.read_csv(join("..", "data", "1.1.resurgence", "labs", "lab_pvals_ITALY.csv"))
sp = pd.read_csv(join("..", "data", "1.1.resurgence", "labs", "lab_pvals_SPAIN.csv"))
us = pd.read_csv(join("..", "data", "1.1.resurgence", "labs", "lab_pvals_USA.csv"))

pdf = fr.append([ge, it, sp, us])
pdf = pdf.drop(columns=['Unnamed: 0'])
# pdf = pdf[pdf.setting != 'never']
pdf = pdf[pdf.day == 0]
pdf.setting = pdf.setting.apply(lambda x: { 'all': 'All Patients', 'ever': 'Ever Severe Patients', 'never': 'Never Severe Patients'}[x])

pdf

# lab = df.Lab.unique().tolist()[0]
labs = ['C-reactive protein (CRP) (Normal Sensitivity)', 'Ferritin', 'D-dimer', 'creatinine', 'albumin']

for l in labs:
    plot = day0_overview(cdf, l, pdf)

    plot = apply_theme(
        plot,
        axis_y_title_font_size=16,
        title_anchor='start',
        # legend_title_orient='left',
        legend_orient='right',
        header_label_font_size=16
    )

    plot.display()
# cdf

In [ ]:
def plot_country_raw(_d, patient_group='all', lab=None):
    d = _d.copy()
    
    """
    DATA PREPROCESSING...
    """
    d['se_top'] = d['mean'] + d['se']
    d['se_bottom'] = d['mean'] - d['se']
    d.wave = d.wave.apply(lambda x: { 'early': 'Early', 'late': 'Late' }[x])
    d = d[d.setting != 'never']
    d = d[d.setting == patient_group]
    d = d.rename(columns={
        'wave': 'Wave'
    })
    d = d[d.country != 'SINGAPORE']
    d = d[d.country != 'GERMANY']
    d.country = d.country.apply(lambda x: x.capitalize() if x != 'USA' else x)
    d = d[d.Lab == lab]
    
    if lab in ['C-reactive protein (CRP) (Normal Sensitivity)', 'D-dimer']:
        d = d[d.scale == 'log']
        yTitle = 'Mean Lab Values (Log)'
    else:
        d = d[d.scale == 'original']
        yTitle = 'Mean Lab Values'

    """
    CONSTANTS
    """
    LABS = d.Lab.unique().tolist()
    WAVE_COLOR = [
        '#D45E00', # '#BA4338', # early
        '#0072B2', # late
        'black'
    ]
    COUNTRY_COLORS = ['#0072B2', '#029F73', '#D45E00', '#CB7AA7']
    
    """
    PLOT
    """
    titleX=-60    
    opacity=0.7
    width=350
    height=250
    size=3
    
    """
    LABS
    """
    # LAB_DROPDOWN = alt.binding_select(options=LABS)
    # LAB_SELECTION = alt.selection_single(fields=["Lab"], bind=LAB_DROPDOWN, init={"Lab": LABS[0]}, name="Select")
    
    line_m = alt.Chart(
        d
    ).mark_line(
        size=size, opacity=opacity, point=True
    ).encode(
        x=alt.X('days_since_positive:N', title='Days Since Positive'),
        y=alt.Y('mean:Q', scale=alt.Scale(zero=False), title=yTitle, axis=alt.Axis(titleX=titleX)),
        color=alt.Color('country:N', scale=alt.Scale(range=COUNTRY_COLORS), title='Country')
    ).properties(
        width=width,
        height=height
    )
    
    """
    ERROR BAR
    """
    error_m = line_m.mark_errorbar(color='gray', opacity=0.3).encode(
        x=alt.X('days_since_positive:N', title='Days Since Positive'),
        y=alt.Y('se_top:Q', scale=alt.Scale(zero=False), title=yTitle, axis=alt.Axis(titleX=titleX)),
        y2=alt.Y2('se_bottom:Q'),
        # color=alt.value('gray')
        # color=alt.Color('country:N', scale=alt.Scale(range=COUNTRY_COLORS))
    )
    
    """
    COMBINE
    """
    line_m = (line_m + error_m).facet(
        column=alt.Column('Wave:N', header=alt.Header(labelOrient="top", title=None, titleOrient="bottom", labels=True))
    )

    plot = (
        alt.hconcat(line_m, spacing=20).resolve_scale(y='shared')
    ).properties(
        title={
            "text": f"Country-Level Lab Values of {'All' if patient_group == 'all' else 'Ever Severe'} Patients by Wave",
            "dx": 80,
            "subtitle": [
                consistent_loinc[lab]
                # lab, #.title(),
                # get_visualization_subtitle(data_release='2021-01-25', with_num_sites=False)
            ], 
            "subtitleFontSize": 16,
            "subtitleColor": "gray",
        }
    )
    
    return plot

In [ ]:
raw = pd.read_csv(join("..", "data", "1.1.resurgence", "labs", "lab_values_bycountry.csv"))

labs = ['C-reactive protein (CRP) (Normal Sensitivity)', 'Ferritin', 'D-dimer', 'creatinine', 'albumin']

patient_groups = ['all', 'ever']

for lab in labs:
    
    for i, patient_group in enumerate(patient_groups):
        if i == 0:
            res = plot_country_raw(raw, patient_group=patient_group, lab=lab)
        else:
            res = alt.vconcat(
                res, plot_country_raw(raw, patient_group=patient_group, lab=lab), spacing=30
            ).resolve_scale(y='shared', color='independent')

    res = apply_theme(
        res,
        axis_y_title_font_size=16,
        header_label_font_size=16,
        title_anchor='start',
        legend_orient='right'
    )

    res.display()

In [ ]:
cdf[(cdf.country == 'ITALY') & (cdf.Lab == 'Ferritin') & (cdf.days_since_positive == 0)]

In [ ]:
obd = pd.read_csv(join("..", "data", "1.1.resurgence", "labs", "lab_obs_bycountry.csv"))

obd

In [ ]:
obd = obd[(obd.cohort == 'dayX') & (obd.setting == 'all') & (obd.day == 0)]

obd

In [ ]:
def obs(obd, lab):
    d = obd.copy()

    """
    DATA PREPROCESSING...
    """
    d.wave = d.wave.apply(lambda x: { 'early': 'Early', 'late': 'Late' }[x])
    d = d.rename(columns={
        'wave': 'Wave'
    })
    d = d[d.country != 'SINGAPORE']
    d = d[d.country != 'GERMANY']
    d.country = d.country.apply(lambda x: x.capitalize() if x != 'USA' else x)

    # "Also, for the lab observations, in germany whenever there’s just a number 0, we’ll count it as unobserved and we should show no arrow for that"
    d = d[~((d.country == "Germany") & (d[lab] == 0))]
    
    d = d.pivot_table(values=lab, index=['country', 'setting'], columns='Wave').reset_index()
    d = d[(d.Early.notna() & d.Late.notna())]
    d['increase'] = d.Early < d.Late

    """
    CONSTANTS
    """
    COUNTRIES = ['France', 'Italy', 'Spain', 'USA']
    COUNTRY_COLORS = ['#0072B2','#029F73', '#D45E00', '#CB7AA7']

    """
    PLOT
    """
    titleX=-80
    opacity=0.7

    """
    LABS
    """    
    bar = alt.Chart(
        d
    ).mark_bar(
        size=6
    ).encode(
        x=alt.X('Early:Q', scale=alt.Scale(zero=False), title='Percentage of Patients'),
        x2=alt.X2('Late:Q'),
        y=alt.Y('country:N', title='Country', axis=alt.Axis(titleX=titleX)),
        color=alt.Color('country:N', title='Country', scale=alt.Scale(domain=COUNTRIES, range=COUNTRY_COLORS))
    ).properties(
        width=300,
        height=320
    )

    tr = alt.Chart(
        d
    ).transform_filter(
        alt.FieldOneOfPredicate(field='increase', oneOf=[True])
    ).mark_point(
        shape="triangle-right", filled=True, size=300, xOffset=-3, opacity=1
    ).encode(
        x=alt.X('Late:Q', scale=alt.Scale(zero=False), title='Percentage of Patients'),
        y=alt.Y('country:N', title='Country', axis=alt.Axis(titleX=titleX)),
        color=alt.Color('country:N', title='Country', scale=alt.Scale(domain=COUNTRIES, range=COUNTRY_COLORS))
    )

    tl = alt.Chart(
        d
    ).transform_filter(
        alt.FieldOneOfPredicate(field='increase', oneOf=[False])
    ).mark_point(
        shape="triangle-left", filled=True, size=300, xOffset=3, opacity=1
    ).encode(
        x=alt.X('Late:Q', scale=alt.Scale(zero=False), title='Percentage of Patients'),
        y=alt.Y('country:N', title='Country', axis=alt.Axis(titleX=titleX), scale=alt.Scale(domain=COUNTRIES)),
        color=alt.Color('country:N', title='Country', scale=alt.Scale(domain=COUNTRIES, range=COUNTRY_COLORS))
    )

    baseline = alt.Chart(
        pd.DataFrame({'baseline': [1]})
    ).mark_rule(color='gray').encode(
        x=alt.X('baseline:Q')
    )

    plot = (bar + tr + tl)

    plot = plot.properties(
        title={
            "text": [
                f"Patients Tested at Day 0 from Early to Late Wave"
            ],
            "dx": 95,
            "subtitle": [
                # 'Relative to Day 0 Mean Value During Early Phase',
                consistent_loinc[lab],
                #get_visualization_subtitle(data_release='2021-01-25', with_num_sites=False)
            ],
            "subtitleFontSize": 16,
            "subtitleColor": "gray",
        }
    )
    return plot

labs = ['C-reactive protein (CRP) (Normal Sensitivity)', 'Ferritin', 'D-dimer', 'creatinine', 'albumin']

for lab in labs:
    res = obs(obd, lab)

    res = apply_theme(
        res,
        axis_y_title_font_size=16,
        title_anchor='start',
        # legend_title_orient='left',
        legend_orient='right',
        header_label_font_size=16
    )

    res.display()